Süper—iki örneği de netçe özetleyeyim, ardından **nedenleri / eksikleri / riskleri** ve **daha sağlam (gelişmiş) bir sürüm** vereyim.

---

# Ne yaptınız? (Adım adım yorum)

## 1) SOM ile keşif ve aykırı adayları bulma

* **Veriyi okudunuz** → `X = dataset.iloc[:, :-1]`, `y = dataset.iloc[:, -1]`
  (Son sütun “başvuru onaylandı mı?” etiketi; görselleştirmede marker/renk için kullanılıyor.)
* **Ölçekleme** → `MinMaxScaler(0,1)` (SOM için önemli; mesafe tabanlı.)
* **SOM eğitimi** → `MiniSom(x=10,y=10,input_len=15,sigma=1.0,lr=0.5)`
  Grid 10×10, Gauss komşuluğu; `train_random(num_iteration=100)` ile online eğitim.
* **U-Matrix görselleştirme** → `som.distance_map()`
  Açık tonlar: komşu prototipler arası mesafe yüksek → **sınır/ayrım/aykırı bölgeler**.
  Üstüne `y` ile marker’lar çizildi (onaylı/onaysız).
* **Aykırı seçimi (manuel)** → `som.win_map(X)` ile BMU koordinatlarındaki örnekleri alıp **elde seçtiğiniz** birkaç hücreyi (ör. `(1,8), (7,8), (8,3), (8,8)`) **fraud** adayı saydınız.

## 2) Zayıf etiketlerle (weak labels) küçük bir ANN

* **Fraud listesi**: seçilen hücrelerdeki örnekler **1**, diğerleri **0** → `is_fraud`.
* **Girdi**: `customers = dataset.iloc[:, 1:]` (ID hariç).
* **Ölçekleme**: `StandardScaler()` (ANN için uygun).
* **ANN**: 2→1 Dense (relu→sigmoid), `binary_crossentropy`.
* **Amaç**: SOM’dan gelen “fraud olasılığı yüksek” örnekleri kullanarak **müşterileri risk puanına göre sıralamak**.

---

# Doğru yaklaşım mı? — Evet ama “eğitici demo” seviyesinde

* **SOM ile keşif** ve **U-Matrix** yorumlama doğru.
* Weak-label (SOM → ANN) akışı da **güzel bir fikir**: SOM aykırılıklarını “risk etiketi” yapıp **rank** üretmek pratikte işe yarar.

**Ama** birkaç önemli eksik / risk var:

### 1) ID ve etiket sütunlarının kullanımı

* İlk örnekte `X = dataset.iloc[:, :-1]` → **ID sütunu da girebilir** (CSV yapısına bağlı).
  → **Öneri**: net olalım:

  ```python
  ids = dataset.iloc[:, 0].values        # CustomerID
  X   = dataset.iloc[:, 1:-1].values     # ID ve son etiket (y) hariç gerçek özellikler
  y   = dataset.iloc[:, -1].values       # Onay etiketi (sadece görselleştirme için)
  input_len = X.shape[1]                 # MiniSom input_len’i sabit 15 yazmayın
  ```
* İkinci kısımda `customers = dataset.iloc[:, 1:]` → **sondaki y etiketi de içeri girmiş olabilir.**
  ANN’e girdi verirken **etiketi** (approval) girdi olarak koymak **sızıntı** (leakage) olabilir.
  → **Öneri**: `customers = dataset.iloc[:, 1:-1].values`.

### 2) Aykırı seçim yöntemi (ad-hoc)

* Koordinasyonları **elde seçmek** (örn. `(1,8)`, `(7,8)` …) **subjektif**.
  → **Öneri (otomatik)**:

  * **BMU mesafesi (quantization error)**: $d_i = \|x_i - w_{bmu(i)}\|$
    `d_i`’leri hesapla, **%95 persentil** üstünü **aykırı** say.
  * **U-Matrix eşiği**: `som.distance_map()`’te **top %5** en açık tonlu hücrelerin tüm örnekleri.
  * **Düşük yoğunluk** (BMU başına çok az örnek) + yüksek U-Matrix beraber.

### 3) Üyelik kontrolü hatası

```python
if dataset.iloc[i,0] in frauds:
    is_fraud[i] = 1
```

* `frauds` bir **2D** array → `in` operatörü **yanlış** (eleman bazlı arar, sütunlarda da eşleşebilir).
  → **Öneri**: fraud ID’leri bir **set** yapın, `if ids[i] in fraud_id_set`.

### 4) Ölçekleme sızıntısı

* ANN kısmında `StandardScaler().fit_transform(customers)` **tüm veri üzerinde** → **data leakage**.
  → **Öneri**: train/val/test ayır, scaler’ı **sadece train** ile fit et.

### 5) Kararlı eğitim

* `MiniSom.train_random` deterministik değil; **seed** ver, ya da `train_batch` kullan.
  → `MiniSom(..., random_seed=42)` veya `som.random_weights_init(X)` + `train_batch`.

### 6) Karışık veri tipleri

* Australian Credit’te **kategorik + sayısal** var. Tek başına MinMax tüm sütunlara uygulandığında, **kategorikler ordinalmiş gibi** davranılır.
  → **Öneri**: One-Hot encode (ya da Gower benzeri karma tip mesafeler; MiniSom saf Öklid kullanıyor).

### 7) Değerlendirme (ground-truth yoksa)

* “Fraud” gerçek etiketi yok; ANN’i aynı veri üzerinde eğitip skorlamak **sadece sıralama** sağlar.
  → **Öneri**: **hold-out** ayır; SOM’u **train** üzerinde kur; weak-label’ı **train** üzerinde üret; ANN’i **train**’de eğit; **test**’te yalnız skorla (auc yoksa en azından dağılım/kalibrasyon grafikleri).

---

# Gelişmiş (daha sağlam) sürüm — **Tam kod**

Aşağıdaki script:

* ID ve y’yi **ayırır**, input\_len’i **dinamik** ayarlar.
* SOM aykırılarını **BMU mesafesi persentili** ile otomatik seçer.
* Fraud ID’lerini **güvenli** çıkarır.
* ANN’i **train/test** ayrımı ve **leakage’sız scaler** ile eğitir.
* MiniSom için **seed** verir.

> `pip install minisom` gerekir.

In [1]:
# som_pipeline_robust.py
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from minisom import MiniSom
import tensorflow as tf
from tensorflow.keras import layers, models

# ==== 0) Veri oku ve sütunları ayır ====
df = pd.read_csv("Credit_Card_Applications.csv")

ids = df.iloc[:, 0].values          # CustomerID (sadece takip için)
y_approval = df.iloc[:, -1].values  # Onay etiketi (0/1) -> sadece görselleştirme/analiz
X = df.iloc[:, 1:-1].values         # Özellikler: ID ve y hariç

# ==== 1) SOM için ölçekleme ====
mm = MinMaxScaler()
X_mm = mm.fit_transform(X)

# ==== 2) SOM eğitimi (kararlı) ====
m, n = 10, 10                 # grid boyutu (isteğe göre ayarla: ~ 5*sqrt(N) nöron)
inp_len = X_mm.shape[1]
som = MiniSom(x=m, y=n, input_len=inp_len, sigma=1.0, learning_rate=0.5, random_seed=42)
som.random_weights_init(X_mm)
som.train_batch(X_mm, num_iteration=200)  # batch eğitimi daha stabil

# ==== 3) BMU ve örnek bazlı mesafeler ====
# Her örnek için: BMU ve BMU'ya olan Öklid mesafesi (quantization distance)
bmu_coords = []
bmu_dists  = []
weights = som.get_weights()   # (m, n, input_len)

for x in X_mm:
    w = som.winner(x)         # (i,j)
    bmu_coords.append(w)
    w_vec = weights[w[0], w[1], :]
    bmu_dists.append(np.linalg.norm(x - w_vec))

bmu_coords = np.array(bmu_coords)   # shape (N, 2)
bmu_dists  = np.array(bmu_dists)    # shape (N,)

# ==== 4) Aykırı seçimi (otomatik, persentil eşiği) ====
p = 95   # top %5
thr = np.percentile(bmu_dists, p)
is_outlier = (bmu_dists >= thr).astype(int)

fraud_idx = np.where(is_outlier == 1)[0]
fraud_ids = set(ids[fraud_idx])

print(f"[INFO] Top {100-p}% aykırı eşiği: {thr:.4f} | Aykırı sayısı: {len(fraud_idx)}")

# ==== 5) Weak-label ile ANN (train/test) ====
# Not: weak label = is_outlier
X_train, X_test, y_train, y_test, id_tr, id_te = train_test_split(
    X, is_outlier, ids, test_size=0.2, random_state=42, stratify=is_outlier
)

# Leakage önleme: scaler yalnızca train'de fit
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc  = sc.transform(X_test)

tf.random.set_seed(42)
ann = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(X_train_sc.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
ann.compile(optimizer='adam', loss='binary_crossentropy')

ann.fit(X_train_sc, y_train, epochs=15, batch_size=16, verbose=0)

# Test skoru (olasılık) — gerçek "fraud" ground-truth yoksa sadece sıralama için kullan
y_score = ann.predict(X_test_sc).ravel()

# Sonuçları CustomerID ile birleştirip en riskli ilk 20'yi yazdır
rank = pd.DataFrame({
    "CustomerID": id_te,
    "weak_label_outlier": y_test,
    "score": y_score
}).sort_values("score", ascending=False)

print("\n[TOP RISK] İlk 20 müşteri (score'a göre):")
print(rank.head(20).to_string(index=False))

[INFO] Top 5% aykırı eşiği: 0.7909 | Aykırı sayısı: 35
5/5 [==============================] - 0s 2ms/step

[TOP RISK] İlk 20 müşteri (score'a göre):
 CustomerID  weak_label_outlier    score
   15699963                   1 0.309224
   15658504                   1 0.261514
   15766663                   0 0.178978
   15622729                   0 0.151382
   15800773                   0 0.126571
   15651460                   1 0.120335
   15813718                   0 0.112216
   15636521                   0 0.109199
   15686461                   0 0.103938
   15573077                   0 0.101540
   15688210                   0 0.099649
   15704509                   0 0.094599
   15585855                   1 0.087011
   15731989                   0 0.086217
   15671769                   0 0.082367
   15791769                   0 0.082172
   15579583                   0 0.081786
   15796569                   0 0.080866
   15599440                   0 0.079697
   15580914                   0

### Bu sürümün artıları

* **ID / y sızıntısı yok:** SOM’e ve ANN’e yalnızca **özellikler** gidiyor.
* **Otomatik aykırı eşiği:** BMU mesafesinin **%95 persentil** üstü.
* **Güvenli ID eşleme:** Fraud ID’leri set olarak çıkarılıyor.
* **Hold-out test:** Scaler fit **yalnızca train**.
* **Daha stabil SOM:** `train_batch`, `random_seed`.

---

# Daha da ileri (opsiyonel geliştirmeler)

* **Grid auto-size:** `m*n ≈ 5*sqrt(N)` kuralı; farklı gridleri deneyip **QE/TE** ile seç.
* **U-Matrix + yoğunluk birlikte:** Yüksek U-Matrix bölgeleri **ve** çok az örnek alan BMU’lar → **daha güçlü aykırı sinyali**.
* **K-Means prototip kümeleme:** `som.get_weights().reshape(m*n, -1)` üzerinde `kmeans++` + **Elbow/Silhouette**.
* **Karma tip veri:** Kategorikler için **One-Hot**; istersen önce **PCA** ile hafif indirgeme (gürültüyü azaltır).
* **PU Learning / Anomaly ensembles:** Weak labels’ı **Positive–Unlabeled** çerçevesinde ele al ya da **IsolationForest/LOF** gibi aykırı modellerle **ensemble** yap.
* **Kalibrasyon:** Skorları **Platt/Isotonic** ile kalibre et (eğer bir miktar ground-truth varsa).
* **Stabilite analizi:** Farklı seed/grid ile **consensus outliers** (kesişim) – daha güvenilir.

---

## Son söz

Yaklaşımınız **öğrenme amaçlı doğru bir iskelet**:
**SOM ile keşfet → aykırı adayları çıkar → onları weak-label yap → MLP ile risk skoru üret.**
Yukarıdaki düzeltmelerle **üretim kalitesine** yaklaşırsınız (sızıntı, ID/etiket ayrımı, aykırı seçiminde otomasyon, hold-out vb.).
İsterseniz, **U-Matrix + yoğunluk** kombinasyonlu otomatik aykırı seçimi ve **prototip K-Means etiketleme** ekleyen bir sürümü de yazabilirim.

## Soru
opsiyonel geliştirmeleri eklemek gerekli mi ? yoksa tüm eğitim bölümlerini bitirdikten sonra tekrar yaparken mi dönelim ? konuları sağlam öğrenmek istiyorum çünkü.

Kısa cevap: **Opsiyonelleri şimdi eklemek zorunda değilsin.** Öncelik, konsepti sağlam oturtmak. Ama **bazı “olmazsa olmaz” düzeltmeleri** şimdiden uygula ki yanlış alışkanlık yerleşmesin. Gelişmiş kısımları kurs bittiğinde ikinci turda ekleriz.

## Şimdi mutlaka yapman gerekenler (küçük ama kritik)

Bunlar hem öğrenmeyi bozmaz hem de hatalı sonuç riskini azaltır:

1. **ID / etiket sızıntısını kes:**

   ```python
   ids = dataset.iloc[:, 0].values
   X   = dataset.iloc[:, 1:-1].values   # ID ve son etiket hariç
   y   = dataset.iloc[:, -1].values
   input_len = X.shape[1]               # MiniSom input_len'i dinamik
   ```
2. **Scaler’ı sadece train ile fit et (ANN tarafı):**

   ```python
   X_tr, X_te, y_tr, y_te = train_test_split(X, is_fraud, test_size=0.2, random_state=42, stratify=is_fraud)
   sc = StandardScaler()
   X_tr = sc.fit_transform(X_tr)
   X_te = sc.transform(X_te)
   ```
3. **Fraud eşleşmesini güvenli yap (set kullan):**

   ```python
   fraud_id_set = set(frauds[:, 0].astype(int))   # inversed değerlerden ID sütununu al
   is_fraud = np.array([1 if int(ids[i]) in fraud_id_set else 0 for i in range(len(ids))])
   ```
4. **Deterministik SOM (tekrar edilebilirlik):**

   ```python
   som = MiniSom(..., random_seed=42)
   # tercihen:
   som.train_batch(X, num_iteration=200)
   ```

> Bu dört madde “temiz temel” sağlar. Eğitim akışını bozmaz, kod karmaşıklaşmaz.

## Sonra (kursu bitirip geri döndüğünde) ekleyebileceklerin

Bunlar kavramlar oturduktan sonra çok değer katar:

* **Aykırı seçimini otomatikleştir:** BMU mesafesi persentil eşiği (örn. %95) + düşük yoğunluk hücreleri.
* **U-Matrix + yoğunluk kombinasyonu:** Hem sınırda hem seyrek bölgeleri işaretle.
* **Prototipleri K-Means ile kümele:** `k-means++`, Elbow/Silhouette ile k seçimi.
* **Kategorik/sayısal karma veri için daha iyi temsil:** One-Hot + (gerekirse) PCA ön indirgeme.
* **Hold-out/tekrar edilebilir deney düzeni:** Farklı seed/grid’lerle “consensus outliers”.
* **PU learning / ensemble:** SOM + IsolationForest/LOF ile birleştir; skorları kalibre et.

## Neden bu sıra?

* **Şimdi:** SOM’in ne yaptığını “gözle gör” (U-Matrix, BMU mantığı). Temel hataları engelle.
* **Sonra:** Kurstaki tüm DL başlıklarını bitirince, **daha rafine aykırılık/segmentasyon** mantığına dönmek çok daha verimli olur. O zaman hem SOM’u hem de ANN/Autoencoder’ı aynı problemde karşılaştırırsın.

İstersen, “şimdi yapılacak minimum düzeltmeler” için kod parçalarını mevcut dosyana **drop-in** olarak ekleyebilirim (senin örneğine bire bir uyarlanmış). Söyle, ekleyeyim.
